In [1]:
import sys
!{sys.executable} -m pip install qdrant-client ollama sqlalchemy


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
import os, json
import pandas as pd
from modules import qdrant
from modules.sqlite import write_df_to_sqlite
from modules.archiver import generate_knowledge
from modules.vectorizer import vectorize_dataframe

file_name = '20251115-mfi-products.csv'
delimiter = ';'
collection_name = 'test_collection'

# Load data, process it, and store in SQLite and Qdrant
def load_data_and_process():
    # Load the CSV file into a DataFrame
    df = pd.read_csv(os.path.join(os.getcwd(),f"input/{file_name}"), delimiter=delimiter)
    return generate_knowledge(df)
"""
    # store the dataframe in sqlite
    res = write_df_to_sqlite(df)
    if not res:
        print("Error storing DataFrame in SQLite.")

    # Vectorize the DataFrame
    df_vec = vectorize_dataframe(df)

    # Store the vectors in Qdrant
    res = qdrant.store_vectors_in_qdrant(df_vec, collection_name=collection_name)
    if not res:
        print("Error storing vectors in Qdrant.")
 """


knowledge_json = load_data_and_process()
print("Completed initialization and data processing.")

TypeError: unhashable type: 'list'

In [ ]:
import ollama
from modules import input, qdrant, vectorizer

models = ['mistral','stablelm2','avr/sfr-embedding-mistral']
model = models[0] # Choose the model to use


# Generate answer based on prompt
def generate_answer(prompt, model='mistral'):
    return ollama.generate(
        model=model,
        prompt=prompt
    )


# Get user input
def get_user_input():
    #
    qry = input.get_user_input("Enter search term")
    if input.validate_search_term(qry):
        # Proceed with search
        pass
    return qry


# Generate SQL query based on user prompt
def agent_sql_query(prompt, knowledge, model='mistral'):

    # Create the agent prompt
    agent_info = f"You are an AI language model assistant. Your task is to find matching products to the {prompt} using only results from a product database with this schema: {knowledge}. Create an SQL query that returns the product name aliases and descriptions of the matching products."
    res = generate_answer(agent_info, model=model)
    return res['response']


# Perform vector search based on user prompt
def agent_vector_search(prompt, knowledge, collection_name, model='mistral'):

    agent_info = f"You are an AI language model assistant. Your task is to find matching products to the {prompt} using only results from a product database that is described by {knowledge}. Create an embedding vector for the search term and use it to search the vector database for the most relevant products."
    search_result = qdrant.perform_search_in_qdrant(
        query = vectorizer.generate_embeddings(agent_info),
        collection_name = collection_name,
        limit = 1
    )
    res = generate_answer(f"Based on the following search result: {search_result[0].payload}, provide a concise answer about the product.", model=model)
    return res['response']


# Main execution
search_query = get_user_input()
answer = agent_sql_query(search_query, knowledge_json, model=model)
print(answer)

 To find matching vegan products from the given database schema, we can use the following SQL query:

```sql
SELECT product_name_alias AS Product_Name, product_description AS Description
FROM Products
WHERE vegan = 2;
```

This query will return the product names aliases and their descriptions for all items in the "Products" table where the 'vegan' attribute is equal to 2 (indicating that they are vegan products).

Assuming the name of your table is "Products", you can replace it with the actual table name if it differs.
